In [56]:
pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=feafb6f1972e0879870fad8888bb9db166af682af542b376442322f1412e2b1b
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [ ]:
pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install pdfminer3k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100 kB 9.4 MB/s 
     |████████████████████████████████| 49 kB 6.7 MB/s 


In [38]:
import requests
import nltk
from bs4 import BeautifulSoup
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

In [ ]:
r = requests.get("https://www.who.int/news-room/fact-sheets/detail/marburg-virus-disease")
r.encoding = 'utf-8'
html = r.text
print(html[:2000])

 <!DOCTYPE html> <html lang="en"> <head>  <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" /> <meta charset="utf-8" /><meta name="viewport" content="width=device-width, initial-scale=1"><title>
	Marburg virus disease
</title> <link rel="shortcut icon" href="/favicon.ico"> <link rel="manifest" href="/manifest.json"> <link rel="apple-touch-icon" href="/apple-touch-icon-precomposed.png"> <meta name="theme-color" content="#007eb4" /><link href="//use.fontawesome.com/releases/v5.0.10/css/all.css" rel="stylesheet" type="text/css" /> <link rel='stylesheet' href='/ResourcePackages/WHO/assets/dist/styles/grid.min.css?v=14.0.7729.14971'>  <link rel='stylesheet' href='/ResourcePackages/WHO/assets/dist/styles/origin.min.css?v=14.0.7729.14971'><link rel='stylesheet' href='/ResourcePackages/WHO/assets/dist/styles/main.min.css?v=14.0.7729.14971'> <link rel='stylesheet' href='/ResourcePackages/WHO/assets/dist/styles/print.min.css?v=14.0.7729.14971' media='print'>
<script>
                

In [32]:
soup = BeautifulSoup(html)
text = soup.text
print(text[0:2000])

   
	Marburg virus disease
       

                var lang = document.documentElement.lang; 
                window.dataLayer = window.dataLayer || []; 
                window.dataLayer.push({'language': lang });
                window.dataLayer.push({
  "pagetype": "factsheet",
  "targetaudience": "general-audience",
  "targetcountry": "global",
  "publishingoffice": "who-headquarters-(hq)",
  "healthtopic": "Marburghaemorrhagicfever"
}); 
                    (function() {
                    function deleteCookie(cookieName,domain) {
                        document.cookie = cookieName + '=;expires=Mon Jan 01 1900 00:00:00 ; path=/; domain=' + domain;
                    }
                    function loadAndTrack(canTrack) {
                        if (!canTrack) {
                            var domain = location.hostname.replace(/^www\./i, '.');
                            //GA
                            deleteCookie('_ga', domain);
                            deleteCookie('_ga

In [55]:
tokenizer = nltk.tokenize.RegexpTokenizer("\w+")

# Tokenize word
word_tokenize = tokenizer.tokenize(text)
print(word_tokenize[:100])

['Marburg', 'virus', 'disease', 'var', 'lang', 'document', 'documentElement', 'lang', 'window', 'dataLayer', 'window', 'dataLayer', 'window', 'dataLayer', 'push', 'language', 'lang', 'window', 'dataLayer', 'push', 'pagetype', 'factsheet', 'targetaudience', 'general', 'audience', 'targetcountry', 'global', 'publishingoffice', 'who', 'headquarters', 'hq', 'healthtopic', 'Marburghaemorrhagicfever', 'function', 'function', 'deleteCookie', 'cookieName', 'domain', 'document', 'cookie', 'cookieName', 'expires', 'Mon', 'Jan', '01', '1900', '00', '00', '00', 'path', 'domain', 'domain', 'function', 'loadAndTrack', 'canTrack', 'if', 'canTrack', 'var', 'domain', 'location', 'hostname', 'replace', 'www', 'i', 'GA', 'deleteCookie', '_ga', 'domain', 'deleteCookie', '_gat', 'domain', 'deleteCookie', '_gid', 'domain', 'Clarity', 'deleteCookie', '_clck', 'domain', 'deleteCookie', '_clsk', 'domain', 'Google', 'Adsense', 'deleteCookie', '_gcl_au', 'domain', 'else', 'Fire', 'GTM', 'Google', 'Tag', 'Manager

In [37]:
words = [token.lower() for token in tokens]
print(words[:100])

['marburg', 'virus', 'disease', 'var', 'lang', 'document', 'documentelement', 'lang', 'window', 'datalayer', 'window', 'datalayer', 'window', 'datalayer', 'push', 'language', 'lang', 'window', 'datalayer', 'push', 'pagetype', 'factsheet', 'targetaudience', 'general', 'audience', 'targetcountry', 'global', 'publishingoffice', 'who', 'headquarters', 'hq', 'healthtopic', 'marburghaemorrhagicfever', 'function', 'function', 'deletecookie', 'cookiename', 'domain', 'document', 'cookie', 'cookiename', 'expires', 'mon', 'jan', '01', '1900', '00', '00', '00', 'path', 'domain', 'domain', 'function', 'loadandtrack', 'cantrack', 'if', 'cantrack', 'var', 'domain', 'location', 'hostname', 'replace', 'www', 'i', 'ga', 'deletecookie', '_ga', 'domain', 'deletecookie', '_gat', 'domain', 'deletecookie', '_gid', 'domain', 'clarity', 'deletecookie', '_clck', 'domain', 'deletecookie', '_clsk', 'domain', 'google', 'adsense', 'deletecookie', '_gcl_au', 'domain', 'else', 'fire', 'gtm', 'google', 'tag', 'manager

In [70]:
tokenizer = nltk.tokenize.RegexpTokenizer("\w+")

# Tokenize sentence
sent_tokenize = tokenizer.tokenize(text)
print(sent_tokenize[:1000])

['Marburg', 'virus', 'disease', 'var', 'lang', 'document', 'documentElement', 'lang', 'window', 'dataLayer', 'window', 'dataLayer', 'window', 'dataLayer', 'push', 'language', 'lang', 'window', 'dataLayer', 'push', 'pagetype', 'factsheet', 'targetaudience', 'general', 'audience', 'targetcountry', 'global', 'publishingoffice', 'who', 'headquarters', 'hq', 'healthtopic', 'Marburghaemorrhagicfever', 'function', 'function', 'deleteCookie', 'cookieName', 'domain', 'document', 'cookie', 'cookieName', 'expires', 'Mon', 'Jan', '01', '1900', '00', '00', '00', 'path', 'domain', 'domain', 'function', 'loadAndTrack', 'canTrack', 'if', 'canTrack', 'var', 'domain', 'location', 'hostname', 'replace', 'www', 'i', 'GA', 'deleteCookie', '_ga', 'domain', 'deleteCookie', '_gat', 'domain', 'deleteCookie', '_gid', 'domain', 'Clarity', 'deleteCookie', '_clck', 'domain', 'deleteCookie', '_clsk', 'domain', 'Google', 'Adsense', 'deleteCookie', '_gcl_au', 'domain', 'else', 'Fire', 'GTM', 'Google', 'Tag', 'Manager

In [64]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [68]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each